In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from transformers import BertTokenizerFast, BertModel, BertConfig, BertForTokenClassification, TextDataset
from transformers import AdamW, PreTrainedTokenizer, PreTrainedTokenizerFast, BertForSequenceClassification,get_cosine_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup, Trainer, DataCollatorForTokenClassification, AdamWeightDecay
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, TensorDataset, SequentialSampler
import torch
from tqdm.notebook import tqdm
import os
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, accuracy_score
import json
from collections import Counter
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import argparse
from dataclasses import dataclass, field
from typing import Optional
import torch.nn as nn
from sklearn.model_selection import  train_test_split
import random
import re
import argparse
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import pickle as pkl

In [2]:
def load_checkpoint_cls(model_name, path):
    
    model = BertForSequenceClassification.from_pretrained(model_name)
    state_dict = torch.load(path)

    mkeys = list(model.state_dict().keys())
    skeys = list(state_dict.keys())


    for k in skeys:

        if k not in mkeys:
            del state_dict[k]

    mkeys = list(model.state_dict().keys())
    skeys = list(state_dict.keys())

    for k in mkeys:

        if k not in skeys:
            state_dict[k] = model.state_dict()[k]


    model.load_state_dict(state_dict)
    
    
    return model


In [3]:
checkpoint = "" # insert path to model checkpoint
model = load_checkpoint_cls("deepset/gbert-large",checkpoint)

Some weights of the model checkpoint at deepset/gbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

In [6]:
data = pd.read_csv("model_input.tsv", sep="\t", index_col=0)

In [31]:
def clean(x):
    
    x = re.sub("\n|\[|\]","",x)
    x = re.sub("\s+"," ",x)
    x = x.split(" ")[1:]
    x = [int(y) for y in x]
    return x

In [41]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [49]:
result = []
for index, row in tqdm(data.iterrows(),total=len(data)):
    x = clean(row["X"])
    output = model(torch.tensor(np.stack([x,x])).to("cuda"))
    label = np.argmax(output["logits"][0].detach().cpu().numpy())
    result.append(label)

Exception ignored in: <function tqdm.__del__ at 0x7fa33fe97ee0>
Traceback (most recent call last):
  File "/home/konle/anaconda3/envs/gpu/lib/python3.8/site-packages/tqdm/std.py", line 1131, in __del__
    self.close()
  File "/home/konle/anaconda3/envs/gpu/lib/python3.8/site-packages/tqdm/notebook.py", line 260, in close
    if self.total and self.n < self.total:
AttributeError: 'tqdm_notebook' object has no attribute 'total'
Exception ignored in: <function tqdm.__del__ at 0x7fa33fe97ee0>
Traceback (most recent call last):
  File "/home/konle/anaconda3/envs/gpu/lib/python3.8/site-packages/tqdm/std.py", line 1131, in __del__
    self.close()
  File "/home/konle/anaconda3/envs/gpu/lib/python3.8/site-packages/tqdm/notebook.py", line 260, in close
    if self.total and self.n < self.total:
AttributeError: 'tqdm_notebook' object has no attribute 'total'
Exception ignored in: <function tqdm.__del__ at 0x7fa33fe97ee0>
Traceback (most recent call last):
  File "/home/konle/anaconda3/envs/gpu/

KeyboardInterrupt: 

In [9]:
frame = pd.DataFrame()
frame["True"] = Y_ML
frame["Prediction_ED"] = result
frame.to_csv(checkpoint+"result.tsv", sep="\t")